In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import os
import time
import shutil

# --- 1. CONFIGURATION ---
BATCH_SIZE = 32
LR = 0.001
EPOCHS = 10 
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Path Setup
PATH_TASK = '/kaggle/input/image-orientation-pretext-task-models/pytorch/default/1'

model_configs = {
    'resnet18': {
        'CropMix':    f'{PATH_TASK}/fullTraining_ResNet18.pth',
        'NoCropMix':  f'{PATH_TASK}/fullTraining_ResNet18_nocropmix.pth',
        'FineTuning': f'{PATH_TASK}/fineTuning_ResNet18.pth',
    },
    'resnet50': {
        'CropMix':    f'{PATH_TASK}/fullTraining_ResNet50.pth',
        'NoCropMix':  f'{PATH_TASK}/fullTraining_ResNet50_nocropmix.pth',
        'FineTuning': f'{PATH_TASK}/fineTuning_ResNet50.pth',
    },
    'efficientnet_b0': {
        'CropMix':    f'{PATH_TASK}/fullTraining_EfficientNetB0.pth',
        'NoCropMix':  f'{PATH_TASK}/fullTraining_EfficientNetB0_nocropmix.pth',
        'FineTuning': f'{PATH_TASK}/fineTuning_EfficientNetB0.pth',
    },
    'efficientnet_b3': {
        'CropMix':    f'{PATH_TASK}/fullTraining_EfficientNetB3.pth',
        'NoCropMix':  f'{PATH_TASK}/fullTraining_EfficientNetB3_nocropmix.pth',
        'FineTuning': f'{PATH_TASK}/fineTuning_EfficientNetB3.pth',
    },
    'inception_v3': {
        'CropMix':    f'{PATH_TASK}/fullTraining_InceptionV3.pth',
        'NoCropMix':  f'{PATH_TASK}/fullTraining_InceptionV3_nocropmix.pth',
        'FineTuning': f'{PATH_TASK}/fineTuning_InceptionV3.pth',
    },
    'simple_CNN':{
        'CropMix':    f'{PATH_TASK}/SimpleRotationCNN.pth',
        'NoCropMix':  f'{PATH_TASK}/SimpleRotationCNN224_nocropmix.pth',
    }
}

VOC_CLASSES = [
    'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 
    'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person', 
    'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor'
]

# --- 2. AUTOMATIC DATASET SETUP ---
def setup_voc_structure():
    input_root = '/kaggle/input/pascal-voc-2012-dataset'
    working_voc_dir = '/kaggle/working/VOCdevkit/VOC2012'
    
    if os.path.exists(working_voc_dir) and os.path.exists(os.path.join(working_voc_dir, 'JPEGImages')):
        print("Dataset structure already exists.")
        return

    print("Setting up VOC dataset structure...")
    source_voc_dir = None
    for root, dirs, files in os.walk(input_root):
        if 'JPEGImages' in dirs:
            source_voc_dir = root
            break
            
    if not source_voc_dir:
        raise FileNotFoundError("Could not find VOC data in input!")

    os.makedirs(working_voc_dir, exist_ok=True)
    
    folders = ['Annotations', 'ImageSets', 'JPEGImages', 'SegmentationClass', 'SegmentationObject']
    for folder in folders:
        src = os.path.join(source_voc_dir, folder)
        dst = os.path.join(working_voc_dir, folder)
        if os.path.exists(src) and not os.path.exists(dst):
            os.symlink(src, dst)
            
    print("Dataset setup complete.")

setup_voc_structure()

# --- 3. DATASET HANDLING & TRANSFORMS ---
class PascalVOC_Classification(datasets.VOCDetection):
    def __init__(self, root, year, image_set, download=False, transform=None):
        super().__init__(root, year, image_set, download=download, transform=transform)
        self.class_to_idx = {name: i for i, name in enumerate(VOC_CLASSES)}

    def __getitem__(self, index):
        img, target = super().__getitem__(index)
        label_vector = torch.zeros(20, dtype=torch.float32)
        objects = target['annotation']['object']
        if not isinstance(objects, list):
            objects = [objects]
        for obj in objects:
            class_name = obj['name']
            if class_name in self.class_to_idx:
                idx = self.class_to_idx[class_name]
                label_vector[idx] = 1.0
        return img, label_vector

# Define transforms
transform_standard = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transform_efficientNetB3 = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transform_inception = transforms.Compose([
    transforms.Resize((299, 299)), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Initialize Dataset
train_dataset = PascalVOC_Classification(
    root='/kaggle/working', 
    year='2012',
    image_set='train',
    download=False,
    transform=transform_standard
)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)


# --- 4. MODEL BUILDER (FIXED FOR INCEPTION) ---
def build_model(architecture, variant, custom_path=None):
    # 1. Determine if we need AuxLogits (Only for Inception)
    use_aux = False
    if architecture == 'inception_v3':
        if variant == 'NoCropMix' or variant == 'ImageNet':
            use_aux = True
    
    # 2. Initialize Base Model
    if variant == 'ImageNet':
        model = getattr(models, architecture)(weights='DEFAULT')
    else:
        if architecture == 'inception_v3':
            model = getattr(models, architecture)(weights=None, aux_logits=use_aux)
        else:
            model = getattr(models, architecture)(weights=None)
    
    # 3. Load Custom Weights
    if variant in ['CropMix', 'NoCropMix', 'FineTuning'] and custom_path:
        if not os.path.exists(custom_path):
            print(f"!! ERROR: File not found {custom_path}. Using Random weights !!")
        else:
            print(f"Loading {variant} weights from disk...")
            checkpoint = torch.load(custom_path, map_location='cpu')
            state_dict = checkpoint.state_dict() if isinstance(checkpoint, nn.Module) else \
                         checkpoint.get('state_dict', checkpoint)
            
            if list(state_dict.keys())[0].startswith('module.'):
                state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}
            
            filtered_dict = {k: v for k, v in state_dict.items() 
                             if not ('fc' in k or 'classifier' in k or 'AuxLogits' in k)}
            try:
                model.load_state_dict(filtered_dict, strict=False)
            except Exception as e:
                print(f"Error loading {variant}: {e}")

    # 4. Replace Head(s) for VOC (20 Classes)
    if 'resnet' in architecture:
        model.fc = nn.Linear(model.fc.in_features, 20)
    elif 'efficientnet' in architecture:
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, 20)
    elif 'inception' in architecture:
        model.fc = nn.Linear(model.fc.in_features, 20)
        # CRITICAL FIX: Resize Aux head if it exists
        if hasattr(model, 'AuxLogits') and model.AuxLogits is not None:
            model.AuxLogits.fc = nn.Linear(768, 20) 

    return model.to(DEVICE)


# --- 5. TRAINING FUNCTION (UPDATED FOR ACCURACY) ---
def train_model(model, arch_name, variant_name):
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9)
    
    loss_history = []
    acc_history = []  # Store accuracy per epoch
    
    model.train()
    print(f"  Training {arch_name} [{variant_name}]...")
    
    for epoch in range(EPOCHS):
        running_loss = 0.0
        running_corrects = 0
        total_elements = 0
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            
            # --- Dynamic Forward Pass ---
            outputs = model(inputs)
            
            # Handle Inception Tuple (AuxLogits)
            if isinstance(outputs, tuple):
                main_out, aux_out = outputs
                loss1 = criterion(main_out, labels)
                loss2 = criterion(aux_out, labels)
                loss = loss1 + 0.4 * loss2
                logits_for_acc = main_out # Calculate accuracy on Main head only
            else:
                loss = criterion(outputs, labels)
                logits_for_acc = outputs
            
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            
            # --- Calculate Accuracy (Element-wise) ---
            # Sigmoid > 0.5 is the standard threshold for Multi-label
            preds = (torch.sigmoid(logits_for_acc) > 0.5).float()
            
            # Count correct elements (Hamming Score numerator)
            # We compare the entire 20-length vector element by element
            running_corrects += (preds == labels).sum().item()
            total_elements += labels.numel() # Batch_Size * 20
        
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = running_corrects / total_elements
        
        loss_history.append(epoch_loss)
        acc_history.append(epoch_acc)

    return loss_history, acc_history


# --- 6. EXECUTION LOOP (UPDATED FOR SUBPLOTS) ---
VARIANTS_TO_TEST = ['CropMix', 'NoCropMix', 'FineTuning', 'ImageNet', 'Random']

for arch, paths in model_configs.items():
    print(f"\n{'='*60}")
    print(f"ARCH: {arch}")
    print(f"{'='*60}")
    
    # 1. Handle Inception/EfficientNet Resize Logic
    if arch == 'inception_v3':
        print(f">> Switching to 299x299 for {arch}")
        train_dataset.transform = transform_inception
    elif arch == 'efficientnet_b3':
        print(f">> Switching to 300x300 for {arch}")
        train_dataset.transform = transform_efficientNetB3
    elif train_dataset.transform != transform_standard:
        print(f">> Switching to 224x224 for {arch}")
        train_dataset.transform = transform_standard

    # 2. Prepare Plot (1 Row, 2 Columns: Loss | Accuracy)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    for variant in VARIANTS_TO_TEST:
        custom_path = paths.get(variant, None)
        
        # Build
        model = build_model(arch, variant, custom_path)
        
        # Train & Record
        start_t = time.time()
        loss_hist, acc_hist = train_model(model, arch, variant)
        duration = time.time() - start_t
        
        # Plot Loss on Left (ax1)
        ax1.plot(loss_hist, label=f"{variant}", marker='.')
        
        # Plot Accuracy on Right (ax2)
        ax2.plot(acc_hist, label=f"{variant}", marker='.')
        
        print(f"    -> {variant} Done ({duration:.0f}s). Final Loss: {loss_hist[-1]:.4f} | Final Acc: {acc_hist[-1]:.4f}")

    # 3. Finalize Plots
    # Loss Plot details
    ax1.set_title(f'{arch} - Loss')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('BCE Loss')
    ax1.legend()
    ax1.grid(True)
    
    # Accuracy Plot details
    ax2.set_title(f'{arch} - Accuracy')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Accuracy (Element-wise)')
    ax2.legend()
    ax2.grid(True)
    
    plt.show()

print("\nAll comparisons complete.")

Dataset structure already exists.

ARCH: resnet18
Loading CropMix weights from disk...
  Training resnet18 [CropMix]...


KeyboardInterrupt: 

<Figure size 1200x600 with 0 Axes>